# Maps


## Variables

* [ ] Temperature, $K$
* [ ] Salinity, $$
* [ ] Currents, [$m s^{-1}$]
* [ ] SSH, [$cm$]
* [ ] MLD, [$m$]
* [ ] Density, [$g cm^{-3}$]
* [ ] Geostrophic Currents, [$m s^{-1}$]
* [ ] Vorticity, [$s^{-1}$]
* [ ] Strain, 
* [ ] Lagrangian Trajectories, [$km$]

## Models

**"Truth"**
* [ ] GLORYS - Reanalysis (Simulation + Observations)
* [ ] GLO12 - Analysis (Forecast + Observations)

**Models**
* Model (x3) (Forecast)
  * [ ] GLO12 (Physical)
  * [ ] GLONET (ML)
  * [ ] XiHe (ML)
  * [ ] WenHai (ML)



In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

from typing import Callable
import cmocean
from matplotlib import ticker
import pandas as pd
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)
from dataclasses import dataclass, field
from utils.types import (
    SeaSurfaceHeight,
    Temperature,
    Salinity,
    ZonalVelocity,
    MeridionalVelocity,
    GeostrophicZonalVelocity,
    GeostrophicMeridionalVelocity,
    MixedLayerDepth,
)



%load_ext autoreload
%autoreload 2

## Base Variable

In [ ]:
import copernicusmarine as cmc
from datetime import datetime

@dataclass
class ReAnalysis:
    name: str = "glorys12"
    color: str = "black"
    linestyle: str = "-"

    def load_data(
        self, variable: str = "zos",
        date: datetime = datetime(2020,1,4),
        depth: float=0.49,
    ):
        depth_min, depth_max = depth - 0.01, depth + 0.01
        data = cmc.open_dataset(
            dataset_id="cmems_mod_glo_phy_myint_0.083deg_P1D-m", #"cmems_mod_glo_phy_my_0.083deg_P1D-m",
            variables=[variable],
            minimum_longitude=-180,
            maximum_longitude=180,
            minimum_latitude=-80,
            maximum_latitude=90,
            minimum_depth=depth_min,
            maximum_depth=depth_max,
            start_datetime=date,
            end_datetime=date
        )
        return data
        



In [ ]:
date = datetime(2024,1,3)
model = ReAnalysis()

variable = "zos"
depth = 0.49
data = model.load_data(variable, date, depth)

In [ ]:
data 

## Loading Variables

## Plotting Code

In [ ]:
from utils.types import generate_wednesdays_in_year
from utils.preprocessing import (
    latlon_deg2m,
    rectilinear_to_regular_grid,
    time_rescale,
    validate_latitude,
    validate_longitude,
    xr_cond_average,
    fuse_base_coords
)

dates = list()
for wednesday in generate_wednesdays_in_year(2024):
    start_datetime = wednesday 
    std_wedn = wednesday.strftime('%Y%m%d')  # Format as YYYYMMDD
    dates.append(std_wedn)

len(dates)

def geoprocess_fn(ds, fill_value: int | float | None = None, lead_time: int = 0, idepth: int = 0):

    try:
        ds = ds.isel(depth=idepth, time=lead_time)
    except ValueError:
        ds = ds.isel(time=lead_time)
        

    # assign time coord correctly
    ds = ds.assign_coords({"time": np.atleast_1d(ds.time)})
    
    # validate coordinates
    ds = validate_longitude(validate_latitude(ds))

    return ds

## Sea Surface Height

In [ ]:
from tqdm.auto import tqdm
from functools import partial
from utils.types import ForecastDataset, DiagnosticDataset
forecast_config = ForecastDataset() # DiagnosticDataset() # 
diagonstic_config = DiagnosticDataset()

In [ ]:
pbar = tqdm(enumerate(get_all_wednesdays(2024)))
for i, wednesday in pbar:
    start_datetime = wednesday 
    std_wedn = wednesday.strftime('%Y%m%d')  # Format as YYYYMMDD
    pbar.set_description(f"Date ({i}): {std_wedn}")

    # FORECAST VARIABLES
    glonet=xr.open_dataset(f"https://minio.dive.edito.eu/project-oceanbench/public/glonet_full_2024/{std_wedn}.zarr",engine="zarr",)
    glo12=xr.open_dataset(f"https://minio.dive.edito.eu/project-oceanbench/public/GLO12/{std_wedn}.zarr",engine="zarr",)
    wenhai=xr.open_dataset(f"https://minio.dive.edito.eu/project-oceanbench/public/WENHAI/{std_wedn}.zarr",engine="zarr",)
    xihe=xr.open_dataset(f"https://minio.dive.edito.eu/project-oceanbench/public/XIHE/{std_wedn}.zarr",engine="zarr",)
    
    # glonet=xr.open_dataset(f"https://minio.dive.edito.eu/project-glonet/public/tmp/GLONET/trajs/{std_wedn}.zarr",engine="zarr",)
    # glonet=xr.open_dataset(f"https://minio.dive.edito.eu/project-glonet/public/tmp/GLONET/process/{std_wedn}.zarr",engine="zarr",)


    

In [ ]:
pbar = tqdm(enumerate(get_all_wednesdays(2024)))
for i, wednesday in pbar:
    start_datetime = wednesday 
    std_wedn = wednesday.strftime('%Y%m%d')  # Format as YYYYMMDD
    pbar.set_description(f"Date ({i}): {std_wedn}")

    # FORECAST VARIABLES
    glonet=xr.open_dataset(f"https://minio.dive.edito.eu/project-oceanbench/public/tmp/GLONET/process/{std_wedn}.zarr",engine="zarr",)
    glo12=xr.open_dataset(f"https://minio.dive.edito.eu/project-oceanbench/public/tmp/GLO12/process/{std_wedn}.zarr",engine="zarr",)
    wenhai=xr.open_dataset(f"https://minio.dive.edito.eu/project-oceanbench/public/tmp/WENHAI/process/{std_wedn}.zarr",engine="zarr",)
    xihe=xr.open_dataset(f"https://minio.dive.edito.eu/project-oceanbench/public/tmp/XIHE/process/{std_wedn}.zarr",engine="zarr",)
    
    # glonet=xr.open_dataset(f"https://minio.dive.edito.eu/project-glonet/public/tmp/GLONET/trajs/{std_wedn}.zarr",engine="zarr",)
    # glonet=xr.open_dataset(f"https://minio.dive.edito.eu/project-glonet/public/tmp/GLONET/process/{std_wedn}.zarr",engine="zarr",)

